# GRT Final

This is the final development notebook of the GRT project.  In this notebook converge all the lessons learned during development.

In [1]:
from gravray import *
from gravray.util import *
from gravray.stats import *
from gravray.spice import *
from gravray.plot import *
from gravray.sampling import *
from gravray.orbit import *

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%matplotlib nbagg

## Initialize

In [3]:
dt=elTime(0)
Spice.loadKernels()
cneos=pd.read_csv("data/cneos_fireball_data_location.csv")
cneos.set_index("Name",inplace=True)
earth=Body("EARTH")

## Example conditions

In [4]:
#Site 
lon=59.8703**Angle.Deg #rad
lat=55.0958*Angle.Deg #rad
alt=23.3*Const.km #m

#Time
datestring="02/15/2013 03:20:34"
fmt="%m/%d/%Y %H:%M:%S"
t=Spice.str2tdb(datestring)

#Population
weights=[0.6]
locs=[
    [0.5,0.5,-2.0],
    [2.0,0.3,-2.6]
]
scales=[
    [1.3,0.7,0.5],
    [0.4,0.9,1.6]
]
angles=[
    [-40.0*Angle.Deg,-86.0*Angle.Deg,0.0*Angle.Deg],
    [+80.0*Angle.Deg,-109.0*Angle.Deg,0.0*Angle.Deg]
]
ranges=[
    [Util.fin2Inf(1e-2,360.0),Util.fin2Inf(360.0-1e-2,360.0)],
    [Util.fin2Inf(1e-2,360.0),Util.fin2Inf(360.0-1e-2,360.0)]
]
NEOs=MultiVariate([1,1,1,0,0])
NEOs.setUnflatten(weights,locs,scales,angles,ranges)

#Direction
Aloc=103.5*Angle.Deg
hloc=18.55*Angle.Deg
vimp=-18.6*Const.km/Const.s

Rdir=np.array([[Aloc],[hloc],[vimp]])

#Input conditions
print(f"Impact conditions:")
print(f"\tDate: {datestring}")
print(f"\tlon. {lon*Angle.Rad:.4g}, lat. {lat*Angle.Rad:.4g}, alt. {alt/Const.km:.4g} km")
print(f"\tAloc = {Aloc*Angle.Rad:.4g}, hloc = {lat*Angle.Rad:.4g}, vimp = {vimp/Const.km} km/s")

Impact conditions:
	Date: 02/15/2013 03:20:34
	lon. 61.54, lat. 55.1, alt. 23.3 km
	Aloc = 103.5, hloc = 55.1, vimp = -18.6 km/s


## Test routines

In [5]:
prob=GrtProb(t,earth,NEOs,verbose=True)
prob.setLocation(lon,lat,alt)
p=prob.calcPdir([Aloc,hloc,vimp])
p,prob.ph,prob.detJ,prob.Rdir

Impact conditions: np.array([[1.806415775814131],[0.32375857624494814],[-18600.0]])
Terminal elements: [  0.72231578   0.5297087    5.18079402 326.57568287 103.0150748
  24.56944643]
Unbound terminal elements: [ 0.89757661  0.11897496 -3.518794    2.27937803 -0.91414209 -2.61391129]


(5.228572267234881e-26,
 5.586619321805399e-05,
 -9.359098886202953e-22,
 [1.806415775814131, 0.32375857624494814, -18600.0])

In [6]:
prob=GrtProb(t,earth,NEOs,verbose=False)
prob.setLocation(lon,lat,alt)
p=prob.calcPsky(np.array([[Aloc],[hloc],[vimp]]))
p,prob.phs,prob.detJs,prob.selements,prob.Rsky

(array([5.22857227e-26]),
 array([5.58661932e-05]),
 array([-9.35909889e-22]),
 array([[  0.72231578,   0.5297087 ,   5.18079402, 326.57568287,
         103.0150748 ,  24.56944643]]),
 array([[ 1.80641578e+00,  3.23758576e-01, -1.86000000e+04]]))

In [7]:
prob=GrtProb(t,earth,NEOs,verbose=False)
p=prob.calcPimp(np.array([[lon],[lat],[alt],[Aloc],[hloc],[vimp]]))
p,prob.phs,prob.detJs,prob.selements,prob.Rimp

(array([5.22857227e-26]),
 array([5.58661932e-05]),
 array([-9.35909889e-22]),
 array([[  0.72231578,   0.5297087 ,   5.18079402, 326.57568287,
         103.0150748 ,  24.56944643]]),
 array([[ 1.07403440e+00,  9.61603114e-01,  2.33000000e+04,
          1.80641578e+00,  3.23758576e-01, -1.86000000e+04]]))

## Test integrations

In [8]:
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="quad")
i,di=pint.integrate({"A":[np.pi],"h":[np.pi/4],"v":np.array([-43.3,-11.1])*Const.km/Const.s})
print(i,di,pint.fun_calls)

3.539251483977331e-23 3.2858412673551106e-23 21


In [9]:
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="fixed_quad")
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-18.6])*Const.km/Const.s})
dt=elTime(1)
print(i,di,pint.fun_calls)

Elapsed time since last call: 256.293 ms
1.060624924834944e-25 None 5


In [11]:
prob=GrtProb(t,earth,NEOs,verbose=False)
prob.setLocation(lon,lat,alt)
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="fixed_quad",opt=dict(n=37))
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-72.2,-11.1])*Const.km/Const.s})
dt=elTime(1)
print(i,di,pint.fun_calls)

Elapsed time since last call: 541.177 ms
2.0715599473990289e-22 None 37


In [12]:
"""
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="quad")
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-18.6])*Const.km/Const.s})
dt=elTime(1)
print(i,di,pint.fun_calls)
pint=MultiQuad(prob.calcPsky,["A","h","v"],integrator="quad")
dt=elTime(0)
i,di=pint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-72.2,-11.1])*Const.km/Const.s})
dt=elTime(1)
print(i,di,pint.fun_calls)
"""
1

1

In [14]:
nint=MultiCube(prob.calcPsky,["A","h","v"])
dt=elTime(0)
i=nint.integrate({"A":[0,2*np.pi],"h":[0,np.pi/2],"v":np.array([-72.2,-11.1])*Const.km/Const.s})
dt=elTime(1)
print(i)

Elapsed time since last call: 392.972 ms
1.6894810998515917e-21


In [15]:
prob=GrtProb(t,earth,NEOs,verbose=False)
prob.setLocation(lon,lat,alt)
NCALLS=0
def f(*X):
    global NCALLS
    A,h,v=X
    p=prob.calcPdir([A,h,v])
    NCALLS+=1
    if (NCALLS%1000)==0:
        print(f"{NCALLS} calls...")
    return p

In [16]:
print(datetime.now())
dt=elTime(0)
i=integrate.nquad(f,[[0,2*np.pi],[0,np.pi/2],np.array([-72.2,-11.1])*Const.km/Const.s],full_output=True)
dt=elTime(1)
print(datetime.now())

2019-08-21 13:39:52.278233


KeyboardInterrupt: 